In [1]:
import cloudscraper as cs
import bs4
import json
import datetime
import pickle
import os

In [ ]:
url = "https://api.opensea.io/api/v1/assets"

for i in range(0, 400):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36','referrer':url, "X-Api-Key": "958b4a41edf5439da174927c48faff89", config["OPENSEA_API_KEY"]}
    querystring = {"token_ids":list(range((i*50)+1, (i*50)+51)),
                   "asset_contract_address":"0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7",
                   "order_direction":"desc",
                   "offset":"0",
                   "limit":"50"}
    response = requests.get(url, params=querystring, headers=headers)
    
    print(i, end=" ")
    if response.status_code != 200:
        print(response.status_code)
        print('error')
        break
    
    #Getting meebits data
    meebits = response.json()['assets']
    #Parsing meebits data
    parsed_meebits = [parse_meebit_data(meebit) for meebit in meebits]
    #storing parsed meebits data into MongoDB
    meebits_collection.insert_many(parsed_meebits)